In [2]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('helper_functions/')
# import util as util
import util

In [3]:
from pymongo import MongoClient

client = MongoClient()

In [4]:
db = client.airbnb


print('The list of all databases currently in the MongoDB client is:')
print(client.list_database_names())

print('\nThe list of all collections in the airbnb database is:')
print(db.list_collection_names())

The list of all databases currently in the MongoDB client is:
['admin', 'airbnb', 'config', 'local', 'test']

The list of all collections in the airbnb database is:
['listings', 'listings_with_calendar', 'reviews', 'listings_with_reviews_and_cal', 'listings_with_reviews_m', 'calendar']


In [99]:
dtype_r = {'listing_id': 'string',
           'id' : 'string',
    'review_id': 'string','date': 'string', 
        'reviewer_id': 'string','reviewer_name': 'string',
           'comments': 'string'}

dtype_l = {'id':'string',
    'name': 'string',
    'host_name': 'string',
    'neighbourhood_group': 'string',
    'host_id' : 'string',
    'neighbourhood': 'string',
    'room_type': 'string',
    'price': 'string',
    'last_review': 'string',
    'reviews_per_month': 'string',
    'license': 'string'
}

reviews_df = pd.read_csv('Data/reviews.csv',dtype=dtype_r, keep_default_na = False)
listings_df = pd.read_csv('Data/listings.csv',dtype=dtype_l,keep_default_na = False)

In [100]:
#First few views
print(listings_df.dtypes)
print('\nThe first few rows of df are:')
display(listings_df.head())

id                                string[python]
name                              string[python]
host_id                           string[python]
host_name                         string[python]
neighbourhood_group               string[python]
neighbourhood                     string[python]
latitude                                 float64
longitude                                float64
room_type                         string[python]
price                             string[python]
minimum_nights                             int64
number_of_reviews                          int64
last_review                       string[python]
reviews_per_month                 string[python]
calculated_host_listings_count             int64
availability_365                           int64
number_of_reviews_ltm                      int64
license                           string[python]
dtype: object

The first few rows of df are:


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,977395984065981849,Home in Brooklyn · 1 bedroom · 1 bed · 1 bath,95344065,Derek,Brooklyn,Sheepshead Bay,40.591790,-73.942850,Private room,30,31,1,1/3/24,0.86,7,339,1,
1,729947657876634696,Rental unit in The Bronx · 1 bedroom · 1 bed ·...,566660,Markus,Bronx,Longwood,40.827373,-73.903147,Private room,29,30,2,12/31/23,0.14,2,34,1,
2,648033676238017128,Rental unit in Bronx · ★4.89 · 1 bedroom · 1 b...,421601513,J Carlos Retals,Bronx,Kingsbridge,40.863940,-73.903120,Private room,30,30,18,12/26/23,0.95,3,359,6,
3,623137142536549768,Rental unit in Brooklyn · 1 bedroom · 1 bed · ...,106442885,Ava,Brooklyn,East New York,40.660730,-73.890500,Entire home/apt,76,30,1,5/19/22,0.05,1,364,0,
4,871990853610302281,Rental unit in Queens · ★New · 1 bedroom · Hal...,484563208,Tricia,Queens,Howard Beach,40.663586,-73.834532,Private room,85,30,0,,,2,89,0,


In [98]:
#First few views
print(reviews_df.dtypes)
print('\nThe first few rows of df are:')
display(reviews_df.head())

listing_id       string[python]
id               string[python]
date             string[python]
reviewer_id      string[python]
reviewer_name    string[python]
comments         string[python]
dtype: object

The first few rows of df are:


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits. <br/>Nous avons a...
1,2595,19176,2009-12-05,53267,Cate,Great experience.
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en..."
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...


In [101]:
importlib.reload(util)

listings_df['last_review'] = pd.to_datetime(listings_df['last_review'], errors='coerce')
reviews_df['date'] = reviews_df['date'].apply(util.convert_date_str_to_datetime)
reviews_df['date'] = reviews_df['date'].apply(lambda x: x if not pd.isnull(x) else None)


C:\Users\Reason\AppData\Local\Temp\ipykernel_5912\2037543876.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  listings_df['last_review'] = pd.to_datetime(listings_df['last_review'], errors='coerce')


In [106]:
listings_df['last_review'].dtype
listings_df['last_review'] = listings_df['last_review'].apply(lambda x: x.isoformat() if not pd.isnull(x) else None)


In [59]:
num_nat_values = listings_df['last_review'].isna().sum()

print(f'There are {num_nat_values} NaT values in the last_review column.')

There are 11500 NaT values in the last_review column.


In [107]:
time1 = datetime.now()
reviews_dict = reviews_df.to_dict('records')
time2 = datetime.now()
print(f'Time to perform this operation was {util.time_diff(time1,time2)} hms.')

time1 = datetime.now()
listings_dict = listings_df.to_dict('records')
time2 = datetime.now()
print(f'Time to perform this operation was {util.time_diff(time1,time2)} hms.')




Time to perform this operation was 0:00:07.449191 hms.
Time to perform this operation was 0:00:00.480232 hms.


In [104]:
print(len(listings_dict))
print(len(reviews_dict))

39202
986810


In [108]:
#inserting in mongo lisitngs

db.listings.drop()

time1 = datetime.now()
result_l = db.listings.insert_many(listings_dict)
time2 = datetime.now()
print(f'\nTime to perform this operation was {util.time_diff(time1,time2)} seconds.')


print(f'\nNumber of docs in db.listings is {db.listings.count_documents({})}')


#Inserting in mongo reviews
db.reviews.drop()
time1 = datetime.now()
result_r = db.reviews.insert_many(reviews_dict)
time2 = datetime.now()
print(f'\nTime to perform this operation was {util.time_diff(time1,time2)} seconds.')
print(f'\nNumber of docs in db.reviews is {db.reviews.count_documents({})}')


Time to perform this operation was 0:00:00.637949 seconds.

Number of docs in db.listings is 39202

Time to perform this operation was 0:00:14.230474 seconds.

Number of docs in db.reviews is 986810


In [122]:
db.listings_with_reviews_m.drop()

db.reviews.create_index('listing_id')

pipeline = [
    {
        '$lookup': {
            'from': 'reviews',
            'localField': 'id',
            'foreignField': 'listing_id',
            'as': 'reviews'
        }
    },
    {
        '$out': 'listings_with_reviews_m'
    }
]

# Execute the aggregation pipeline
db.listings.aggregate(pipeline)


print(db.list_collection_names())


['listings', 'listings_with_calendar', 'reviews', 'listings_with_reviews_m', 'calendar', 'listings_with_reviews_and_cal']


In [123]:
print(db.listings_with_reviews_m.count_documents({}))

39202


In [5]:
docs = list(db.listings_with_reviews_m.find({'id': {'$regex': '^111'}}))
output = []

for doc in docs:
    output.append(util.clean_listings_w_date(doc))

print(len(output))

28


In [6]:
dir = 'CHECK'
filename = 'listings_with_reviews_m_subset_1000.json'
util.write_dict_to_dir_json(output, dir, filename)